In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

C:\Users\srijithbalakrishnan\AppData\Local\Temp\ipykernel_3948\1890965923.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import os
import time
import infrarisk.experiments.shelby_e2lcid.shelby_simulations as sim
import infrarisk.src.socioeconomic.se_analysis as se_analysis
import copy
from pathlib import Path
import multiprocessing
import pandas as pd

c:\Users\srijithbalakrishnan\miniconda3\envs\infrarisk\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.4-CAPI-1.17.4) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
NETWORK_DIR = Path("../../data/networks/shelby")
DEPENDENCY_FILE = NETWORK_DIR/"dependencies.csv"
SE_DIR = NETWORK_DIR/'gis/se_data'
SIM_STEP  = 7200

RESILIENCE_LEVEL = 'low'

SCENARIOS_DIR = NETWORK_DIR/"scenarios"

In [4]:
shelby_simulation_original = sim.FullSimulation(network_dir = NETWORK_DIR, 
                                                dependency_file = DEPENDENCY_FILE, 
                                                scenarios_dir = SCENARIOS_DIR, 
                                                se_dir = SE_DIR,
                                                sim_step = SIM_STEP,
                                               robust_level = ['low'],
                                               resource_levels = ['low'],
                                               redundancy_level = ['low'])
shelby_simulation_original.generate_network()

SIM_NUMBER = 50

Water network successfully loaded from ..\..\data\networks\shelby\water/water.inp. The analysis type is set to PDA.
initial simulation duration: 7200s; hydraulic time step: 7200s; pattern time step: 3600s

Loading water service area details...


numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



Power system successfully loaded from ..\..\data\networks\shelby\power\power.json. Single phase power flow simulation will be used.

Loading power service area details...
Transportation network successfully loaded from ..\..\data\networks\shelby\transportation\reduced. Static traffic assignment method will be used to calculate travel times.
Successfully added power network to the integrated graph...
Successfully added water network to the integrated graph...
Successfully added transportation network to the integrated graph...
Integrated graph successfully created.
Generating betweenness centrality...


Loading BokehJS ...

In [5]:
year, tract, county, state = 2017, '*', 157, 47
county = 157
se_dir = NETWORK_DIR/"gis/se_data"
if not os.path.exists(se_dir):
    os.makedirs(se_dir)

ShelbySE = se_analysis.SocioEconomicTable(name = 'Shelby', year = year, 
                                               tract = tract, state = state, 
                                               county = county, dir = se_dir)

ShelbySE.download_se_data(force_download = False)
ShelbySE.create_setable()
shelby_simulation_original.shelby_se = ShelbySE
ShelbySE.plot_interactive(type = "annual receipts")

interactive(children=(Dropdown(description='Industry', layout=Layout(width='max-content'), options=('Accommoda…

In [6]:
from joblib import Parallel, delayed

import multiprocessing
import copy
import random
from IPython.display import clear_output  # Only if using Jupyter/IPython
import dill  

def run_simulations(shelby_simulation_original):
    try:
        compon_count_dict = {
            "water": random.randint(0, 20),
            "power": random.randint(0, 20),
            "transpo": random.randint(0, 20),
        }
        shelby_simulation = copy.deepcopy(shelby_simulation_original)

        shelby_simulation.generate_random_disruptions(compon_count_dict=compon_count_dict)
        repair_order_dict = shelby_simulation.generate_repair_order_dict()
        shelby_simulation.perform_shelby_simulation()

        clear_output(wait=True)
    except StopIteration:
        print("StopIteration encountered")
    except TypeError:
        print("TypeError encountered")
    except AttributeError:
        print("AttributeError encountered")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return None  # Explicit return for failed cases

nb_cpus = 9
SIM_NUMBER = 50

if __name__ == '__main__':
    list1 = [shelby_simulation_original] * SIM_NUMBER
    results = Parallel(n_jobs=nb_cpus)(delayed(run_simulations)(x) for x in list1)

KeyboardInterrupt: 